In [1]:
from pyspark.sql import functions as f
from pyspark.sql import types as t
from pyspark.sql.window import Window
import dateutil.parser

In [2]:
countryCode = 'es'

In [3]:
universe = spark.table('dev_derived_ouniverse.output_total_universe_' + countryCode).withColumn('d_buyer',f.when(f.col("operatorConcatId").isNotNull(),f.lit(1)).otherwise(f.lit(0)))
relevantProducts  =  spark.table('data_user_hien.valueTier_opr_cuEanCode_' + countryCode).where(f.col('cuEanCode').isNotNull())
productAssumptions  =  spark.table('data_user_hien.valueTier_productAssumptions_' + countryCode)
productAssumptionsNoRelevantProducts  = productAssumptions.select(f.avg(f.col('avgTotalProductPrice')).alias('avgTotalProductPriceNoProducts'))
menu      =  spark.table("lead_management.es_business_product_mapping_long_20190829")
productDetails = spark.table('data_sifu.sifu_product_details').where(f.lower(f.col('countryCode')) =='es')

In [4]:
sale  =  spark.table("data_model.ohub_operator_sales").where(f.col('countryCode') == 'ES').where(f.col('transactionDate').isNotNull())

def dateConverter(isoDate):
  dateOutput = dateutil.parser.parse(isoDate).strftime('%y-%m-%d')
  return dateOutput

dateConverterUDF = f.udf(lambda x: dateConverter(x))
sale = sale.where(f.col('transactionDate').rlike("[0-9]"))
sale = sale.withColumn('date', dateConverterUDF(f.col('transactionDate')))
sale = sale.where(f.col('date') >= '19-10-01')

operatorSalesValue = sale.where(f.col('amount') > 0).where(f.col('cuEanCode').isNotNull()).groupBy('operatorOhubId').agg(f.sum(f.col('quantityOfUnits')*((f.col('cuListingPriceInCents')/f.lit(100)).cast('double'))).alias('operatorSalesValue'), f.countDistinct(f.col('cuEanCode')).alias('grab'))

##### product to menu fit

In [6]:
display(relevantProducts)

operatorID,osmid,placeIdGoogle,operatorOhubID,name,address,postalCode,city,latitude,longitude,businessType,website,phone,nameGoogle,addressGoogle,postalCodeGoogle,cityGoogle,latitudeGoogle,longitudeGoogle,businessTypeGoogle,websiteGoogle,type,uid,user_sid,nameOSM,addressOSM,postalCodeOSM,cityOSM,latitudeOSM,longitudeOSM,businessTypeOSM,websiteOSM,cuisineTypeOSM,phoneOSM,nameOHUB,addressOHUB,zipcodeOHUB,cityOHUB,operatorConcatID,channelOHUB,source,globalListChannels,globalChannel,cuisineType,chain,d_buyer,cuEanCode,rationale,productName
/ / 0089343a-76b1-30e4-8c32-1f1402535df1,null,null,0089343a-76b1-30e4-8c32-1f1402535df1,COMERCIAL GUSSTET,HOLANDA,08228,Les Fonts (terrassa),null,null,Otros,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,COMERCIAL GUSSTET,HOLANDA,08228,Les Fonts (terrassa),ES~GESCON~22260294,Otros,OHUB,[Other],Other,us,Unknown,1,8410127127900,Sales Data,"maizena harina fina de maiz espesante sin gluten caja 2,5 kg"
/ / 0089343a-76b1-30e4-8c32-1f1402535df1,null,null,0089343a-76b1-30e4-8c32-1f1402535df1,COMERCIAL GUSSTET,HOLANDA,08228,Les Fonts (terrassa),null,null,Otros,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,COMERCIAL GUSSTET,HOLANDA,08228,Les Fonts (terrassa),ES~GESCON~22260294,Otros,OHUB,[Other],Other,us,Unknown,1,8712100410703,Sales Data,knorr caldo líquido concentrado de carne sin gluten botella 1l
/ / 02a07098-7f85-3fb9-9370-2be1801f97dc,null,null,02a07098-7f85-3fb9-9370-2be1801f97dc,CAFETERIA CARLOS,MANCHA 0002,03183,Torrevieja,null,null,Restaurante menú/carta,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,CAFETERIA CARLOS,MANCHA 0002,03183,Torrevieja,ES~GESCON~30093909,Restaurante menú/carta,OHUB,"[Other, Restaurant, Cafe]",Restaurant,Unknown,Unknown,1,8410127183012,Sales Data,knorr bovril caldo concentrado de carne bote 500g
/ / 02a07098-7f85-3fb9-9370-2be1801f97dc,null,null,02a07098-7f85-3fb9-9370-2be1801f97dc,CAFETERIA CARLOS,MANCHA 0002,03183,Torrevieja,null,null,Restaurante menú/carta,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,CAFETERIA CARLOS,MANCHA 0002,03183,Torrevieja,ES~GESCON~30093909,Restaurante menú/carta,OHUB,"[Other, Restaurant, Cafe]",Restaurant,Unknown,Unknown,1,8712566192038,Sales Data,knorr salsa bechamel deshidratada bote 715g
/ / 041ef817-1a82-4dec-bbb6-d62d1b47b856,null,null,041ef817-1a82-4dec-bbb6-d62d1b47b856,CAF. BADEN BADEN,MAYOR 0013,50740,Fuentes De Ebro,null,null,Bar cafetería,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,CAF. BADEN BADEN,MAYOR 0013,50740,Fuentes De Ebro,ES~GESCON~406218,Bar cafetería,OHUB,"[Other, Cafe, Bar]",Cafe,Unknown,Unknown,1,8712100453373,Sales Data,sirope de chocolate carte d'or botella 1l
/ / 04fe5d7b-cedb-34d3-9a4e-69c6f3c09cb8,null,null,04fe5d7b-cedb-34d3-9a4e-69c6f3c09cb8,MANUEL VAL GONZALEZ,NOVO MESOIRO 0005,15190,Feans De A Coruña,null,null,Otros,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,MANUEL VAL GONZALEZ,NOVO MESOIRO 0005,15190,Feans De A Coruña,ES~GESCON~13346067,Otros,OHUB,[Other],Other,Unknown,Unknown,1,8410127011056,Sales Data,mayonesa hellmann's original cubo 5l sin gluten
/ / 0e54668a-c291-39a8-8d85-3f70ec2b8bf2,null,null,0e54668a-c291-39a8-8d85-3f70ec2b8bf2,MIGUEL SERRANO CHAMORRO,KASUNE 0056,48990,Andra Mari (getxo),null,null,Restaurante menú/carta,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,MIGUEL SERRANO CHAMORRO,KASUNE 0056,48990,Andra Mari (getxo),ES~GESCON~12052635,Restaurante menú/carta,OHUB,"[Other, Restaurant]",Restaurant,Unknown,Unknown,1,8712100453373,Sales Data,sirope de chocolate carte d'or botella 1l
/ / 1129bbbc-8cc3-426a-81b3-5fd4aac